In [12]:
import pandas as pd

Week of September 29th, exploring python filters
imported course catalog dataset

Week of October 6th, exploring python filters
delete unnecesary columns from main dataframe
concatenate GPA dataset

Week of October 13th, added RMP column cleaning to send to next teams

In [55]:
#import datasets

#import course catalog dataset
df = pd.read_csv('2025-sp.csv') #df for data frame

#import course GPA dataset
df_GPA = pd.read_csv('uiuc-gpa-dataset.csv')

#import teacher rating dataset
df_Rate = pd.read_csv('uiuc-tre-dataset.csv')

In [23]:
#modify name function modify_Name(str) to concat name to last name, first initial
def modify_Name(name):
    if not isinstance(name, str):
        return name  # Handle non-string values
    
    parts = name.split(',')
    last_name = parts[0].strip() # Get the last name
    first_initial = parts[1].strip()[:1].upper()  # Get the first initial
    return last_name + ', ' + first_initial

def modify_Name2(name):
    if not isinstance(name, str):
        return name  # Handle non-string values
    
    # Split the name by comma
    parts = name.split(',')
    if len(parts) != 2:
        return name  # Return as-is if format is unexpected
    
    last_name = parts[0].strip()
    first_name = parts[1].strip().split()[0]  # Take only the first name
    
    # Combine in "First Last" format
    return f"{first_name} {last_name}"


In [24]:
#df cleaning and modification (main)

#delete unnecesary columns from main dataframe
drop_cols = ['Term', 'Section Info','Schedule Information', 'Section Title', 'Enrollment Status','Status Code', 'Section Status', 'Section Credit Hours']
df.drop(drop_cols, axis=1, inplace=True) 
df['Primary Instructor (Concat)'] = df['Instructors'].apply(lambda x: str(x).split(';')[0]) #add column for primary instructor
df.rename(columns={'Type Code': 'Sched Type'}, inplace=True) #rename column to match GPA dataset

In [25]:
#df_GPA cleaning and modification
# Chaging the last name to only first initial (to match main DF)
df_GPA['Primary Instructor (Concat)'] = df_GPA['Primary Instructor'].apply(lambda name: modify_Name(name))
df_GPA['Primary Instructor'] = df_GPA['Primary Instructor'].apply(lambda name: modify_Name2(name))

# create mean_df_profBased_GPA + mean_df_classBased_GPA; mean GPA by professor for each class / only by class
group_cols_wProf = ['Subject', 'Number', 'Sched Type', 'Primary Instructor', 'Primary Instructor (Concat)']
group_cols = ['Subject', 'Number', 'Sched Type']
merge_cols = ['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+','C', 'C-', 'D+', 'D', 'D-', 'F', 'W', 'Students']
# by prof + class
mean_df_profBased_GPA = df_GPA.groupby(group_cols_wProf, as_index = False)[merge_cols].mean().round(2)
mean_df_profBased_GPA['Mean Grade By Professor (A+..F,W,Students)'] = mean_df_profBased_GPA[merge_cols].values.tolist()
mean_df_profBased_GPA.drop(merge_cols, axis=1, inplace=True) 
# by only class
mean_df_classBased_GPA = df_GPA.groupby(group_cols, as_index = False)[merge_cols].mean().round(2)
mean_df_classBased_GPA['Mean Grade By Class (A+..F,W,Students)'] = mean_df_classBased_GPA[merge_cols].values.tolist()
mean_df_classBased_GPA.drop(merge_cols, axis=1, inplace=True)

Soft Preference Different Approaches:
Right now, the issue with the merged dataset is that the df_GPA contains GPA information for the past 15 years, starting from 2010 to 2015.
A couple possibilites here:
    1. only take GPA data from the current year
        - missing out on a lot of data here
    2. take the most recent GPA
        - again, missing out on data here 
        - (info could be provided in interface (i.e. data from 2022))
    3. take the average of the GPA's for each professor (CURRENT APPROACH)
        - changing class structure could be inacurate representation

In [56]:
#df_Rate cleaning and modification

#only consider data from past 10 years, drop role, term columns
df_Rate['Year'] = df_Rate['term'].str.extract(r'(\d{4})').astype(float) #extracts the year from term
df_Rate = df_Rate[df_Rate['Year'] >= 2015] #only consider data from past 10 years (20 total bc 2 semesters per year)
#df_Rate.drop(columns=['Year', 'term', "role"], inplace=True)

#seperate Excellent and Outstanding ratings into binary columns 
#DF TEAM SHOULD FIGURE OUT HOW EXCELLENT AND OUTSTANDING ARE RATED
df_Rate['Excellent'] = df_Rate['ranking'].apply(lambda x: 1 if str(x) == 'Excellent' else 0)
df_Rate['Outstanding'] = df_Rate['ranking'].apply(lambda x: 1 if str(x) == 'Outstanding' else 0)

#merge lname and fname columns to match main df primary instructor concat column
df_Rate['Primary Instructor (Concat)'] = df_Rate['lname'].str.title().str.strip() + ', ' + df_Rate['fname'].str.title().str.strip()

#aggregate all excellent and outstanding with same unit, course number, and primary instructor (concat)
df_Rate = (df_Rate.groupby(['unit', 'course', 'Primary Instructor (Concat)'], as_index=True)[['Excellent', 'Outstanding']].sum()).reset_index()

#drop unnecessary columns before merge, rename columns to match main df
#df_Rate.drop(columns=['unit', 'lname', "fname", "ranking", "Excellent", "Outstanding"], inplace=True)
df_Rate.rename(columns={'course': 'Number'}, inplace=True)

#df_Rate was originally a String, convert to int to match main df, create NaN for "???"
df_Rate['Number'] = pd.to_numeric(df_Rate['Number'], errors='coerce')

In [57]:
df_Rate[(df_Rate['Number'] == 100) & (df_Rate['Primary Instructor (Concat)'] == "Kim, G")].head()

,unit,Number,Primary Instructor (Concat),Excellent,Outstanding
3509,Asian American Studies,100.0,"Kim, G",2,0
27943,PSYCHOLOGY,100.0,"Kim, G",1,0


Rating Different Approaches:

^^ rating also provides a tricky challenge - how do you define if the ranknig for a professor is good? does it count indefinitely after you get a ranking? or is it better to get more rankings? wouldnt that put teachers who are new at a disadvatange? is outstanding double as good as excellent? How do you quantify this?

In [2]:
#df merge w/ df_GPA + df_Rate
group_cols_wProf = ['Subject', 'Number', 'Sched Type', 'Primary Instructor (Concat)'] # get rid of full name one bc OG df doesnt have that column
df = df.merge(mean_df_profBased_GPA, how='left', on=group_cols_wProf)
df = df.merge(mean_df_classBased_GPA, how='left', on=group_cols)
df.columns
#df = df.merge(df_Rate, how='left', on=['Number', "Primary Instructor (Concat)"]) 
#note that we are merging only on number and instructor, so we are betting on the fact that one professor only teaches one class of a given course number
#i.e. one professor isnt teaching both CS120 & ECE120 bc if so, they are getting the same rating for both classes...

NameError: name 'df' is not defined

In [19]:
#RMP integration functions. full name search: getRMP(str) & fuzzy name search: getRMPfuzzy(str)

import requests
import re
import json

def getRMP(professor_full_name: str) -> float:
  url = "https://www.ratemyprofessors.com/search/professors/1112?q=" + professor_full_name
  
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
  }
  
  try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
  except requests.exceptions.RequestException as e:
    print(f"Error making request: {e}")
    return None

  match = re.search(r'window\.__RELAY_STORE__ = (.*?);', response.text)
  if not match:
    print("Could not find professor data on the page.")
    return None

  try:
    data = json.loads(match.group(1))
  except json.JSONDecodeError:
    print("Failed to parse JSON data.")
    return None

  for key, value in data.items():
    if isinstance(value, dict) and value.get('__typename') == 'Teacher':
      first_name = value.get('firstName', '')
      last_name = value.get('lastName', '')
      
      if f"{first_name} {last_name}".lower() == professor_full_name.lower():
        avg_rating = value.get('avgRating')
        if avg_rating is not None:
          # Return the float value directly
          return avg_rating
          
  return None

def getRMPfuzzy(professor_full_name: str) -> float:
  url = "https://www.ratemyprofessors.com/search/professors/1112?q=" + professor_full_name
  
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
  }
  
  try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
  except requests.exceptions.RequestException as e:
    print(f"Error making request: {e}")
    return None

  match = re.search(r'window\.__RELAY_STORE__ = (.*?);', response.text)
  if not match:
    print("Could not find professor data on the page.")
    return None

  try:
    data = json.loads(match.group(1))
  except json.JSONDecodeError:
    print("Failed to parse JSON data.")
    return None

  for key, value in data.items():
    if isinstance(value, dict) and value.get('__typename') == 'Teacher':
      avg_rating = value.get('avgRating')
      if avg_rating is not None:
          # Return the float value directly
        return avg_rating
          
  return None

In [1]:
#if you want to save the modified dataframe to a csv file, uncomment the following line (CSV already uploaded in google drive!)
#df.to_csv('modified-2025-sp.csv', index=False)
df.columns

NameError: name 'df' is not defined

DO NOT RUN THE FOLLOWING CODE ON YOUR LAPTOP UNLESS YOU WANT YOUR CODE TO RUN FOR AT LEAST 3 HOURS!
*if you just want the csv, it is uploaded in the google drive. 

In [21]:
from functools import lru_cache
from tqdm import tqdm #need to type "pip3 install tqdm" in terminal if not installed
import pandas as pd

# cached functions to avoid repeated queries
@lru_cache(maxsize=None)
def cached_getRMP(name):
    return getRMP(name)

@lru_cache(maxsize=None)
def cached_getRMPfuzzy(name):
    return getRMPfuzzy(name)

# prepare an empty list to store results
rmp_results = []

chunk_size = 1000  # number of rows per chunk
num_chunks = (len(df) + chunk_size - 1) // chunk_size  # total number of chunks

for i in tqdm(range(num_chunks), desc="Processing RMP"):
    start_idx = i * chunk_size
    end_idx = min((i+1) * chunk_size, len(df))
    chunk = df.iloc[start_idx:end_idx]

    # apply the function to the chunk
    chunk_results = chunk.apply(
        lambda row: (
            cached_getRMP(str(row['Primary Instructor']).strip())
            if pd.notna(row['Primary Instructor']) and str(row['Primary Instructor']).strip() != ''
            else cached_getRMPfuzzy(str(row.get('Primary Instructor (Concat)', '')).strip())
        ),
        axis=1
    )

    rmp_results.extend(chunk_results)

# assign results back to the DataFrame
df['RMP'] = rmp_results

Processing RMP:   0%|          | 0/25 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
df.to_csv('modified+RMP-2025-sp.csv', index=False)